In [2]:
from transformers import AutoConfig, AutoTokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
import streamlit as st
import pandas as pd
from transformers import AutoConfig
from datetime import datetime
import spacy
import dateparser
from sklearn.model_selection import train_test_split
import pandas as pd
import re
from datasets import load_dataset
import calendar
from datetime import date
import torch
from fuzzywuzzy import process
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
from sentence_transformers import SentenceTransformer, util

C:\Users\ДНС\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ДНС\AppData\Roaming\Python\Python310\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
nlp = spacy.load("ru_core_news_sm")

In [ ]:
data = pd.read_csv("dataset.csv")

def create_dataset(row):
    if pd.notnull(row["client"]):
        return {"text": row["client"], "label": 0}
    elif pd.notnull(row["service"]):
        return {"text": row["service"], "label": 1}
    elif pd.notnull(row["management"]):
        return {"text": row["management"], "label": 2}
    else:
        return None

dataset = data.apply(create_dataset, axis=1).dropna()
texts = [item["text"] for item in dataset if item is not None]
labels = [item["label"] for item in dataset if item is not None]

df = pd.DataFrame({"text": texts, "label": labels})
df.to_csv("prepared_dataset.csv", index=False)

In [4]:
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=3)

dataset = Dataset.from_pandas(pd.read_csv("prepared_dataset.csv"))

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/rugpt3small_based_on_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 498/498 [00:00<00:00, 1161.96 examples/s]


In [47]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

C:\Users\ДНС\AppData\Roaming\Python\Python310\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ДНС\AppData\Local\Temp\ipykernel_22620\817690833.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [48]:
trainer.train()

  4%|▍         | 10/250 [00:53<18:22,  4.59s/it]

{'loss': 0.8678, 'grad_norm': 21.24707794189453, 'learning_rate': 1.9200000000000003e-05, 'epoch': 0.2}


  8%|▊         | 20/250 [01:39<18:08,  4.73s/it]

{'loss': 0.475, 'grad_norm': 29.340612411499023, 'learning_rate': 1.8400000000000003e-05, 'epoch': 0.4}


 12%|█▏        | 30/250 [02:25<17:15,  4.71s/it]

...

{'eval_loss': 0.35640957951545715, 'eval_runtime': 15.425, 'eval_samples_per_second': 6.483, 'eval_steps_per_second': 0.843, 'epoch': 5.0}


100%|██████████| 250/250 [22:53<00:00,  5.02s/it]

{'train_runtime': 1373.4728, 'train_samples_per_second': 1.449, 'train_steps_per_second': 0.182, 'train_loss': 0.25546302723884584, 'epoch': 5.0}


100%|██████████| 250/250 [22:53<00:00,  5.50s/it]


TrainOutput(global_step=250, training_loss=0.25546302723884584, metrics={'train_runtime': 1373.4728, 'train_samples_per_second': 1.449, 'train_steps_per_second': 0.182, 'total_flos': 129996307169280.0, 'train_loss': 0.25546302723884584, 'epoch': 5.0})

In [49]:
model.save_pretrained("./finetuned_rugpt3small")
tokenizer.save_pretrained("./finetuned_rugpt3small")

('./finetuned_rugpt3small\\tokenizer_config.json',
 './finetuned_rugpt3small\\special_tokens_map.json',
 './finetuned_rugpt3small\\vocab.json',
 './finetuned_rugpt3small\\merges.txt',
 './finetuned_rugpt3small\\added_tokens.json',
 './finetuned_rugpt3small\\tokenizer.json')

In [15]:
from transformers import pipeline

classifier = pipeline("text-classification", model="./finetuned_rugpt3small", tokenizer="./finetuned_rugpt3small")

result = classifier("Акционерное общество Уральская фольга")
print(result)
result = classifier("1С: Документооборот")
print(result)

[{'label': 'LABEL_0', 'score': 0.9384876489639282}]
[{'label': 'LABEL_1', 'score': 0.7501918077468872}]


In [ ]:
def correct_month(input_month):
    """
    Исправляет возможные опечатки в названии месяца.
    """
    month_mapping = {
        "январь": "01", "января": "01",
        "февраль": "02", "февраля": "02",
        "март": "03", "марта": "03",
        "апрель": "04", "апреля": "04",
        "май": "05", "мая": "05",
        "июнь": "06", "июня": "06",
        "июль": "07", "июля": "07",
        "август": "08", "августа": "08",
        "сентябрь": "09", "сентября": "09",
        "октябрь": "10", "октября": "10",
        "ноябрь": "11", "ноября": "11",
        "декабрь": "12", "декабря": "12"
    }

    corrected_month = process.extractOne(input_month.lower(), month_mapping.keys())
    return month_mapping.get(corrected_month[0]) if corrected_month else None


def extract_date(text):
    doc = nlp(text)
    dates = [ent.text for ent in doc.ents if ent.label_ == "DATE"]
    if dates:
        for date_str in dates:
            date_obj = dateparser.parse(date_str)
            if date_obj:
                return date_obj.strftime("%Y-%m-%d")

    regex_patterns = {
        "interval": r'\bс\s+([а-яА-Я]+)\s+(\d{2})\s+по\s+([а-яА-Я]+)\s+(\d{2})\b',
        "single_date": [
            r'\b(\d{1,2})\.(\d{1,2})\.(\d{2,4})\b',
            r'\b(\d{1,2})\s+([а-яА-Я]+)\s+(\d{2,4})\b',
            r'\b(\d{1,2})\s+([а-яА-Я]+)\s+(\d{2})\s+года\b',
            r'\b(\d{1,2})\s+([а-яА-Я]+)\b'
        ],
        "month_year": r'\b(?<!\d\s)([а-яА-Я]+)\s+(\d{2,4})\b'
    }

    # Обработка интервала дат
    interval_match = re.search(regex_patterns["interval"], text, re.IGNORECASE)
    if interval_match:
        start_month_name, start_year, end_month_name, end_year = interval_match.groups()
        start_month = correct_month(start_month_name)
        end_month = correct_month(end_month_name)
        if start_month and end_month:
            if len(start_year) == 2:
                start_year = f"20{start_year}"
            if len(end_year) == 2:
                end_year = f"20{end_year}"
            start_date_obj = date(int(start_year), int(start_month), 1)
            last_day = calendar.monthrange(int(end_year), int(end_month))[1]
            end_date_obj = date(int(end_year), int(end_month), last_day)
            return f"{start_date_obj.strftime('%Y-%m-%d')} - {end_date_obj.strftime('%Y-%m-%d')}"

    # Обработка одиночных дат
    for pattern in regex_patterns["single_date"]:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            if len(match.groups()) == 3:
                day, month_or_name, year = match.groups()
                month = correct_month(month_or_name)
                if month:
                    if len(year) == 2:
                        year = f"20{year}"
                    return f"{year}-{month.zfill(2)}-{day.zfill(2)}"
            elif len(match.groups()) == 2:  # Обработка дня и месяца без года
                day, month_or_name = match.groups()
                month = correct_month(month_or_name)
                if month:
                    year = datetime.now().year  # Используем текущий год
                    return f"{year}-{month.zfill(2)}-{day.zfill(2)}"

    # Обработка месяца и года
    month_year_match = re.search(regex_patterns["month_year"], text, re.IGNORECASE)
    if month_year_match:
        month_name, year = month_year_match.groups()
        month = correct_month(month_name)
        if month:
            if len(year) == 2:
                year = f"20{year}"
            start_date_obj = date(int(year), int(month), 1)
            last_day = calendar.monthrange(int(year), int(month))[1]
            end_date_obj = date(int(year), int(month), last_day)
            return f"{start_date_obj.strftime('%Y-%m-%d')} - {end_date_obj.strftime('%Y-%m-%d')}"

    date_obj = dateparser.parse(text)
    if date_obj:
        return date_obj.strftime("%Y-%m-%d")

    return None

In [4]:
text = "Найди данные за 23 октября"
print(extract_date(text))

2024-10-23


In [34]:
examples = [
    "Покажи круговой график за 8 октября 24 года",
    "Отобрази данные расходов за 08.10.24",
    "Продажи с июля 23 по сентябрь 23",
    "Сделай анализ за март 2023",
    "График с мая 22 по август 23",
    "График за октябри 22 года",
]

for text in examples:
    print(f"Текст: {text}")
    print(f"Найденная дата/интервал: {extract_date(text)}\n")

Текст: Покажи круговой график за 8 октября 24 года
Найденная дата/интервал: 2024-10-08

Текст: Отобрази данные расходов за 08.10.24
Найденная дата/интервал: 2024-01-08

Текст: Продажи с июля 23 по сентябрь 23
Найденная дата/интервал: 2023-07-01 - 2023-09-30

Текст: Сделай анализ за март 2023
Найденная дата/интервал: 2023-03-01 - 2023-03-31

Текст: График с мая 22 по август 23
Найденная дата/интервал: 2022-05-01 - 2023-08-31

Текст: График за октябри 22 года
Найденная дата/интервал: 2022-10-01 - 2022-10-31



In [42]:
texts = [
    "8 октябри 2024",
    "с июли 23 по сентябрtt 23",
    "31 декабраап 22",
    "февфыль 2024",
    "февфыль 2025",
]

for text in texts:
    print(f"Текст: {text}")
    print(f"Найденная дата: {extract_date(text)}\n")

Текст: 8 октябри 2024
Найденная дата: 2024-10-08

Текст: с июли 23 по сентябрtt 23
Найденная дата: 2023-07-01 - 2023-07-31

Текст: 31 декабраап 22
Найденная дата: 2022-12-31

Текст: февфыль 2024
Найденная дата: 2024-02-01 - 2024-02-29

Текст: февфыль 2025
Найденная дата: 2025-02-01 - 2025-02-28



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

data = pd.read_csv("prepared_dataset.csv")

train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

model_name = "sberbank-ai/ruBert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

def preprocess_data(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokenized["label"] = examples["label"]
    return tokenized


train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 100/100 [00:00<00:00, 6250.36 examples/s]


In [78]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

C:\Users\ДНС\AppData\Roaming\Python\Python310\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [80]:
trainer.train()

model.save_pretrained("./model")
tokenizer.save_pretrained("./model")

  9%|▉         | 7/75 [01:13<11:52, 10.48s/it]
                                               
 33%|███▎      | 25/75 [03:22<06:10,  7.42s/it]

{'eval_loss': 0.299873411655426, 'eval_runtime': 12.05, 'eval_samples_per_second': 8.299, 'eval_steps_per_second': 1.079, 'epoch': 1.0}


                                               
 67%|██████▋   | 50/75 [06:47<03:06,  7.46s/it]

{'eval_loss': 0.20822711288928986, 'eval_runtime': 12.639, 'eval_samples_per_second': 7.912, 'eval_steps_per_second': 1.029, 'epoch': 2.0}


                                               
100%|██████████| 75/75 [10:29<00:00,  8.40s/it]


{'eval_loss': 0.2280440479516983, 'eval_runtime': 11.493, 'eval_samples_per_second': 8.701, 'eval_steps_per_second': 1.131, 'epoch': 3.0}
{'train_runtime': 629.8436, 'train_samples_per_second': 1.896, 'train_steps_per_second': 0.119, 'train_loss': 0.27893513997395836, 'epoch': 3.0}


('./model\\tokenizer_config.json',
 './model\\special_tokens_map.json',
 './model\\vocab.txt',
 './model\\added_tokens.json',
 './model\\tokenizer.json')

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from transformers import pipeline

data = pd.read_csv("prepared_dataset.csv")

classifier = pipeline("text-classification", model="./model", tokenizer="./model")
semantic_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

data['embedding'] = data['text'].apply(lambda x: semantic_model.encode(x, convert_to_tensor=True))

C:\Users\ДНС\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd
from transformers import pipeline

data = pd.read_csv("prepared_dataset.csv")

classifier = pipeline("text-classification", model="./model", tokenizer="./model")

def find_filters_with_classifier(text):
    filters = []
    label_map = {0: "client", 1: "service", 2: "management"}

    for _, row in data.iterrows():
        prediction = classifier(row["text"])[0]
        predicted_label = int(prediction["label"].replace("LABEL_", ""))

        if row["text"].lower() in text.lower():
            formatted_text = row["text"].replace(" ", "")
            filters.append(f"?{label_map[predicted_label]}={formatted_text}")
    
    return filters


def extract_entities_with_classifier(text):
    
    filters = find_filters_with_classifier(text)
    
    dashboards_data = get_dashboards_and_charts()
    
    dashboard_id, chart_id = find_dashboard_and_chart(text, dashboards_data)
    
    extracted_date = extract_date(text)

    entities = {
        "dashboard": dashboard_id,
        "chart": chart_id,
        "filters": filters,
        "date": extracted_date
    }
    return entities 

In [ ]:
def find_filters_with_classifier(text):
    filters = []
    label_map = {0: "client", 1: "service", 2: "management"}

    query_embedding = semantic_model.encode(text, convert_to_tensor=True)

    for _, row in data.iterrows():
        similarity_score = util.cos_sim(query_embedding, row['embedding']).item()

        if similarity_score > 0.7:
            prediction = classifier(row["text"])[0]
            predicted_label = int(prediction["label"].replace("LABEL_", ""))
            formatted_text = row["text"].replace(" ", "")
            filters.append(f"?{label_map[predicted_label]}={formatted_text}")

    return filters

In [4]:
def extract_entities_with_classifier(text):
    filters = find_filters_with_classifier(text)
    
    extracted_date = extract_date(text)
    
    if extracted_date:
        filters.append(f"?date={extracted_date}")
    
    dashboards_data = get_dashboards_and_charts()
    
    dashboard_id, chart_id = find_dashboard_and_chart(text, dashboards_data)
    
    entities = {
        "dashboard": dashboard_id,
        "chart": chart_id,
        "filters": filters
    }
    return entities

In [ ]:
query = "Найди график 'Частота клиентов' на дашборде 'Project EN+' для клиента Акционерное общество Уральская фольга с 1С: Документооборот за 23 октября 24 года"
# Покажи кол-во запросов по *управление* за период
# По запросу понять как называется график
entities = extract_entities_with_classifier(query)

print(entities)

{'dashboard': 13, 'chart': 382, 'filters': ['?service=1С:УправлениеХолдингом.Эн+', '?service=СопровождениеплатформыСЭД', '?date=2024-10-23']}


In [2]:
import requests

url = 'http://localhost:8088'

username = 'admin'
password = 'admin'

auth_response = requests.post(f'{url}/api/v1/security/login', json={
    'username': username,
    'password': password,
    'provider': 'db'
})

if auth_response.status_code == 200:
    auth_token = auth_response.json()['access_token']
    print("Аутентификация прошла успешно.")
else:
    print(f"Ошибка аутентификации: {auth_response.status_code} - {auth_response.text}")
    exit()

headers = {
    'Authorization': f'Bearer {auth_token}'
}

def get_dashboards_and_charts():
    dashboards_response = requests.get(f'{url}/api/v1/dashboard/', headers=headers)
    if dashboards_response.status_code == 200:
        dashboards = dashboards_response.json()['result']
        dashboards_data = {}
        for dashboard in dashboards:
            dashboard_id = dashboard['id']
            dashboard_title = dashboard['dashboard_title']
            dashboards_data[dashboard_title] = {'id': dashboard_id, 'charts': []}
            
            charts_response = requests.get(f"{url}/api/v1/dashboard/{dashboard_id}/charts", headers=headers)
            if charts_response.status_code == 200:
                charts = charts_response.json()['result']
                for chart in charts:
                    dashboards_data[dashboard_title]['charts'].append({
                        'id': chart['id'],
                        'slice_name': chart['slice_name']
                    })
            else:
                print(f"Ошибка при получении графиков: {charts_response.status_code} - {charts_response.text}")
        return dashboards_data
    else:
        print(f"Ошибка при получении дашбордов: {dashboards_response.status_code} - {dashboards_response.text}")
        return None

# Функция для поиска дашборда и графика по текстовому запросу
def find_dashboard_and_chart(query, dashboards_data):
    query_lower = query.lower()
    matched_dashboard = None
    matched_dashboard_id = None
    matched_chart = None
    matched_chart_id = None

    for dashboard_title, dashboard_info in dashboards_data.items():
        if dashboard_title.lower() in query_lower:
            matched_dashboard = dashboard_title
            matched_dashboard_id = dashboard_info['id']
            for chart in dashboard_info['charts']:
                if chart['slice_name'].lower() in query_lower:
                    matched_chart = chart['slice_name']
                    matched_chart_id = chart['id']
                    break
            break

    return matched_dashboard_id, matched_chart_id

Аутентификация прошла успешно.


In [ ]:
import requests

# Укажите URL вашего экземпляра Superset
url = 'http://localhost:8088'

# Укажите свои учетные данные
username = 'xx'
password = 'xx'

# Аутентификация и получение токена доступа
auth_response = requests.post(f'{url}/api/v1/security/login', json={
    'username': username,
    'password': password,
    'provider': 'db'
})

if auth_response.status_code == 200:
    auth_token = auth_response.json()['access_token']
    print("Аутентификация прошла успешно.")
else:
    print(f"Ошибка аутентификации: {auth_response.status_code} - {auth_response.text}")
    exit()

# Заголовки для последующих запросов
headers = {
    'Authorization': f'Bearer {auth_token}'
}

# Получение списка дашбордов
dashboards_response = requests.get(f'{url}/api/v1/dashboard/', headers=headers)

if dashboards_response.status_code == 200:
    dashboards = dashboards_response.json()
    print("Список дашбордов:")
    for dashboard in dashboards['result']:
        print(f"- {dashboard['dashboard_title']} (ID: {dashboard['id']})")
        # Получение графиков для каждого дашборда
        charts_response = requests.get(f"{url}/api/v1/dashboard/{dashboard['id']}/charts", headers=headers)
        if charts_response.status_code == 200:
            charts = charts_response.json()['result']
            print("  Графики:")
            for chart in charts:
                print(f"    - {chart['slice_name']} (ID: {chart['id']})")
        else:
            print(f"  Ошибка при получении графиков: {charts_response.status_code} - {charts_response.text}")
else:
    print(f"Ошибка при получении дашбордов: {dashboards_response.status_code} - {dashboards_response.text}")

Аутентификация прошла успешно.
Список дашбордов:
- Project EN+ (ID: 13)
  Графики:
    - Частота клиентов (ID: 382)
- COVID Vaccine Dashboard (ID: 11)
  Графики:
    - Vaccine Candidates per Phase (ID: 114)
    - Vaccine Candidates per Phase (ID: 115)
    - Vaccine Candidates per Country & Stage (ID: 116)
    - Vaccine Candidates per Country & Stage (ID: 117)
    - Vaccine Candidates per Country (ID: 118)
    - Vaccine Candidates per Country (ID: 119)
    - Vaccine Candidates per Approach & Stage (ID: 120)
- FCC New Coder Survey 2018 (ID: 10)
  Графики:
    - Work Location Preference (ID: 93)
    - Top 15 Languages Spoken at Home (ID: 94)
    - ✈️ Relocation ability (ID: 95)
    - Preferred Employment Style (ID: 96)
    - Location of Current Developers (ID: 98)
    - How much do you expect to earn? ($0 - 100k) (ID: 100)
    - How do you prefer to work? (ID: 101)
    - Highest degree held (ID: 102)
    - First Time Developer & Commute Time (ID: 104)
    - First Time Developer? (ID: 105)

In [4]:
# entities_extractor.py

import re
import calendar
from datetime import date, datetime, timedelta
import pandas as pd
import requests
import spacy
import dateparser
from fuzzywuzzy import process
from transformers import pipeline

# Загрузка модели Spacy для русского языка
nlp = spacy.load("ru_core_news_sm")

# Инициализация классификатора
classifier = pipeline("text-classification", model="./model", tokenizer="./model")

# Загрузка подготовленного датасета
data = pd.read_csv("prepared_dataset.csv")

# Класс для управления аутентификацией и токеном API
class SupersetClient:
    def __init__(self, superset_url, username, password):
        self.superset_url = superset_url
        self.username = username
        self.password = password
        self.token = None
        self.token_expiry = None  # Время истечения токена
        self.headers = None

    def authenticate(self):
        auth_response = requests.post(f'{self.superset_url}/api/v1/security/login', json={
            'username': self.username,
            'password': self.password,
            'provider': 'db'
        })

        if auth_response.status_code == 200:
            auth_data = auth_response.json()
            self.token = auth_data['access_token']
            # Предположим, что токен действует 1 час
            self.token_expiry = datetime.now() + timedelta(hours=1)
            self.headers = {
                'Authorization': f'Bearer {self.token}'
            }
            print("Аутентификация успешна.")
        else:
            raise Exception(f"Ошибка аутентификации: {auth_response.status_code} - {auth_response.text}")

    def ensure_authenticated(self):
        # Проверяем, действителен ли токен
        if not self.token or datetime.now() >= self.token_expiry:
            print("Токен истек или отсутствует. Выполняется повторная аутентификация.")
            self.authenticate()

    def get(self, endpoint):
        self.ensure_authenticated()
        response = requests.get(f'{self.superset_url}{endpoint}', headers=self.headers)
        # Если токен истек или недействителен, повторяем аутентификацию
        if response.status_code == 401:
            print("Токен недействителен. Повторная аутентификация.")
            self.authenticate()
            response = requests.get(f'{self.superset_url}{endpoint}', headers=self.headers)
        return response

# Функция для исправления опечаток в названиях месяцев
def correct_month(input_month):
    month_mapping = {
        "январь": "01", "января": "01",
        "февраль": "02", "февраля": "02",
        "март": "03", "марта": "03",
        "апрель": "04", "апреля": "04",
        "май": "05", "мая": "05",
        "июнь": "06", "июня": "06",
        "июль": "07", "июля": "07",
        "август": "08", "августа": "08",
        "сентябрь": "09", "сентября": "09",
        "октябрь": "10", "октября": "10",
        "ноябрь": "11", "ноября": "11",
        "декабрь": "12", "декабря": "12"
    }

    corrected_month = process.extractOne(input_month.lower(), month_mapping.keys())
    return month_mapping.get(corrected_month[0]) if corrected_month else None

# Функция для извлечения даты из текста
def extract_date(text):
    doc = nlp(text)
    dates = [ent.text for ent in doc.ents if ent.label_ == "DATE"]
    if dates:
        for date_str in dates:
            date_obj = dateparser.parse(date_str)
            if date_obj:
                return date_obj.strftime("%Y-%m-%d")

    regex_patterns = {
        "interval": r'\bс\s+([а-яА-Я]+)\s+(\d{2})\s+по\s+([а-яА-Я]+)\s+(\d{2})\b',
        "single_date": [
            r'\b(\d{1,2})\.(\d{1,2})\.(\d{2,4})\b',
            r'\b(\d{1,2})\s+([а-яА-Я]+)\s+(\d{2,4})\b',
            r'\b(\d{1,2})\s+([а-яА-Я]+)\s+(\d{2})\s+года\b'
        ],
        "month_year": r'\b(?<!\d\s)([а-яА-Я]+)\s+(\d{2,4})\b'
    }

    # Обработка интервала дат
    interval_match = re.search(regex_patterns["interval"], text, re.IGNORECASE)
    if interval_match:
        start_month_name, start_year, end_month_name, end_year = interval_match.groups()
        start_month = correct_month(start_month_name)
        end_month = correct_month(end_month_name)
        if start_month and end_month:
            if len(start_year) == 2:
                start_year = f"20{start_year}"
            if len(end_year) == 2:
                end_year = f"20{end_year}"
            start_date_obj = date(int(start_year), int(start_month), 1)
            last_day = calendar.monthrange(int(end_year), int(end_month))[1]
            end_date_obj = date(int(end_year), int(end_month), last_day)
            return f"{start_date_obj.strftime('%Y-%m-%d')} - {end_date_obj.strftime('%Y-%m-%d')}"

    # Обработка одиночных дат
    for pattern in regex_patterns["single_date"]:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            if len(match.groups()) == 3:
                day, month_or_name, year = match.groups()
                month = correct_month(month_or_name)
                if month:
                    if len(year) == 2:
                        year = f"20{year}"
                    return f"{year}-{month.zfill(2)}-{day.zfill(2)}"

    # Обработка месяца и года
    month_year_match = re.search(regex_patterns["month_year"], text, re.IGNORECASE)
    if month_year_match:
        month_name, year = month_year_match.groups()
        month = correct_month(month_name)
        if month:
            if len(year) == 2:
                year = f"20{year}"
            start_date_obj = date(int(year), int(month), 1)
            last_day = calendar.monthrange(int(year), int(month))[1]
            end_date_obj = date(int(year), int(month), last_day)
            return f"{start_date_obj.strftime('%Y-%m-%d')} - {end_date_obj.strftime('%Y-%m-%d')}"

    date_obj = dateparser.parse(text)
    if date_obj:
        return date_obj.strftime("%Y-%m-%d")

    return None

# Функция для получения списка дашбордов и графиков из Superset
def get_dashboards_and_charts(superset_client):
    dashboards_response = superset_client.get('/api/v1/dashboard/')
    if dashboards_response.status_code == 200:
        dashboards = dashboards_response.json()['result']
        dashboards_data = {}
        for dashboard in dashboards:
            dashboard_id = dashboard['id']
            dashboard_title = dashboard['dashboard_title']
            dashboards_data[dashboard_title] = {'id': dashboard_id, 'charts': []}

            charts_response = superset_client.get(f"/api/v1/dashboard/{dashboard_id}/charts")
            if charts_response.status_code == 200:
                charts = charts_response.json()['result']
                for chart in charts:
                    dashboards_data[dashboard_title]['charts'].append({
                        'id': chart['id'],
                        'slice_name': chart['slice_name']
                    })
            else:
                print(f"Ошибка при получении графиков: {charts_response.status_code} - {charts_response.text}")
        return dashboards_data
    else:
        print(f"Ошибка при получении дашбордов: {dashboards_response.status_code} - {dashboards_response.text}")
        return None

# Функция для поиска дашборда и графика по текстовому запросу
def find_dashboard_and_chart(query, dashboards_data, similarity_threshold=0.5, fuzz_threshold=50):
    query_lower = query.lower()
    lemmatized_query = lemmatize_text(query)  # Лемматизация запроса
    query_embedding = semantic_model.encode(lemmatized_query, convert_to_tensor=True)

    matched_dashboard_id = None
    matched_chart_id = None

    max_dashboard_score = 0
    max_chart_score = 0

    for dashboard_title, dashboard_info in dashboards_data.items():
        lemmatized_dashboard_title = lemmatize_text(dashboard_title)  # Лемматизация названия дашборда

        # Проверяем схожесть с дашбордом
        semantic_similarity = util.cos_sim(query_embedding, semantic_model.encode(lemmatized_dashboard_title, convert_to_tensor=True)).item()
        fuzzy_score = fuzz.token_set_ratio(query_lower, dashboard_title.lower())

        # Средняя метрика (можно настроить веса)
        dashboard_score = (semantic_similarity * 100 + fuzzy_score) / 2

        if dashboard_score > max_dashboard_score and dashboard_score >= fuzz_threshold:
            max_dashboard_score = dashboard_score
            matched_dashboard_id = dashboard_info['id']

            # Поиск графиков внутри текущего дашборда
            for chart in dashboard_info['charts']:
                lemmatized_chart_name = lemmatize_text(chart['slice_name'])  # Лемматизация названия графика

                # Проверяем схожесть с графиком
                chart_semantic_similarity = util.cos_sim(query_embedding, semantic_model.encode(lemmatized_chart_name, convert_to_tensor=True)).item()
                chart_fuzzy_score = fuzz.token_set_ratio(query_lower, chart['slice_name'].lower())

                # Средняя метрика
                chart_score = (chart_semantic_similarity * 100 + chart_fuzzy_score) / 2

                if chart_score > max_chart_score and chart_score >= fuzz_threshold:
                    max_chart_score = chart_score
                    matched_chart_id = chart['id']

    return matched_dashboard_id, matched_chart_id

semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

data['embedding'] = data['text'].apply(lambda x: semantic_model.encode(x, convert_to_tensor=True))

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

data['lemmatized_text'] = data['text'].apply(lemmatize_text)

def find_filters_with_classifier(text, similarity_threshold=0.6, fuzz_threshold=60):
    filters = []
    label_map = {0: "client", 1: "service", 2: "management"}

    # Лемматизация запроса
    lemmatized_query = lemmatize_text(text)

    # Вычисляем эмбеддинг для лемматизированного текста запроса
    text_embedding = semantic_model.encode(lemmatized_query, convert_to_tensor=True)

    for _, row in data.iterrows():
        # Используем лемматизированный текст строки
        lemmatized_row_text = row['lemmatized_text']

        # Вычисляем семантическую схожесть
        similarity = util.cos_sim(text_embedding, row['embedding']).item()

        if similarity >= similarity_threshold:
            # Проверяем нечеткую схожесть
            fuzzy_score = fuzz.token_set_ratio(lemmatized_row_text.lower(), lemmatized_query.lower())
            if fuzzy_score >= fuzz_threshold:
                # Классифицируем текст строки
                prediction = classifier(row["text"])[0]
                predicted_label = int(prediction["label"].replace("LABEL_", ""))
                filters.append(f"?{label_map[predicted_label]}={row['text']}")

    return filters

# Основная функция для извлечения сущностей
def extract_entities(text, superset_url, superset_username, superset_password):
    # Создаем экземпляр клиента Superset
    superset_client = SupersetClient(superset_url, superset_username, superset_password)

    # Получение данных о дашбордах и графиках
    dashboards_data = get_dashboards_and_charts(superset_client)

    # Извлечение дашборда и графика
    dashboard_id, chart_id = find_dashboard_and_chart(text, dashboards_data)

    # Извлечение фильтров
    filters = find_filters_with_classifier(text)

    # Извлечение даты
    extracted_date = extract_date(text)

    # Добавление даты в фильтры
    if extracted_date:
        if ' - ' in extracted_date:
            # Обработка интервала дат
            start_date, end_date = extracted_date.split(' - ')
            filters.append(f"?start_date={start_date}")
            filters.append(f"?end_date={end_date}")
        else:
            filters.append(f"?date={extracted_date}")

    # Формирование результата
    entities = {
        "dashboard_id": dashboard_id,
        "chart_id": chart_id,
        "filters": filters,
    }
    return entities

In [ ]:
from fuzzywuzzy import fuzz
import re

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

def normalize_text(text):
    replacements = [
        r"\b(акционерное общество|ао|общество с ограниченной ответственностью|ооо)\b",
        r"\b(публичное акционерное общество|пао)\b"
    ]
    for pattern in replacements:
        text = re.sub(pattern, "", text, flags=re.IGNORECASE)
    return text.strip()

def correct_spelling(text, vocabulary, threshold=80):
    corrected_words = []
    for word in text.split():
        # Ищем наиболее похожее слово в словаре
        match = process.extractOne(word, vocabulary, scorer=fuzz.token_set_ratio)
        if match and match[1] >= threshold:
            corrected_words.append(match[0])
        else:
            corrected_words.append(word) 
    return " ".join(corrected_words)

# Нормализуем данные, добавляем лемматизацию
data['normalized_text'] = data['text'].apply(normalize_text)
data['lemmatized_text'] = data['normalized_text'].apply(lemmatize_text)

# Создаём словарь из данных для проверки опечаток
vocabulary = set(data['normalized_text'].str.split().sum())

def find_filters_with_classifier(text, similarity_threshold=0.5, fuzz_threshold=90, spelling_threshold=80):
    filters = []
    label_map = {0: "client", 1: "service", 2: "management"}

    normalized_query = normalize_text(text)
    corrected_query = correct_spelling(normalized_query, vocabulary, spelling_threshold)
    print(f"Corrected Query: {corrected_query}")

    lemmatized_query = lemmatize_text(corrected_query)

    text_embedding = semantic_model.encode(lemmatized_query, convert_to_tensor=True)

    for _, row in data.iterrows():
        lemmatized_row_text = row['lemmatized_text']

        similarity = util.cos_sim(text_embedding, row['embedding']).item()

        if similarity >= similarity_threshold:
            fuzzy_score = fuzz.token_set_ratio(lemmatized_row_text.lower(), lemmatized_query.lower())
            if fuzzy_score >= fuzz_threshold:
                prediction = classifier(row["text"])[0]
                predicted_label = int(prediction["label"].replace("LABEL_", ""))
                filters.append(f"?{label_map[predicted_label]}={row['text']}")

    return filters

print(find_filters_with_classifier("Найди график 'Частота клиентов' на дашборде 'Project EN+' для клиента уральская фольга с 1С: Документооборот за 23 октября 24 года"))


Corrected Query: Найди график 'Частота клиентов' на дашборде Project En+ для клиента Уральский фольга с 1С: 1С:Документооборот за 23 октября 24 год
['?client=Акционерное общество Уральская фольга', '?service=1С: Документооборот']


In [75]:
query = "Найди график 'Частота клиентов' на дашборде 'Project EN+' для клиента уральская фольга с 1С: Документооборот за 23 октября 24 года"
superset_url = 'http://localhost:8088/'
superset_username = 'admin'
superset_password = 'admin'

entities = extract_entities(query, superset_url, superset_username, superset_password)

print(entities)

Токен истек или отсутствует. Выполняется повторная аутентификация.
Аутентификация успешна.
{'dashboard_id': 13, 'chart_id': 382, 'filters': ['?service=1С: Документооборот', '?service=1С:Документооборот КВСУ', '?date=2024-10-23']}
